In [2]:
import os
import boto3
import sagemaker
from datetime import date
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
import utils
import pandas as pd
from production.pipelines_code.pipeline import get_pipeline as pipeline_definition
import datetime
from calendar import monthrange

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
# Specify the path you want to change to
new_directory = '/root/NPS forecasting model/nps_client_model/Client simulation/Simulation pipeline'

# Change the current working directory
os.chdir(new_directory)

# Verify the change
current_directory = os.getcwd()
print("Current Directory:", current_directory)


Current Directory: /root/NPS forecasting model/nps_client_model/Client simulation/Simulation pipeline


In [4]:
# Reemplaza esto con la ruta correcta a tu archivo
data_df = pd.read_excel('develop/Marco simulations/Third_Simulations.xlsx', sheet_name='Third_Simulations')
data_df

,filtered_name,S_Satisfaction_label_aux,Model,Cabin,survey_maritz_name,S_Satisfaction_label
0,Aircraft interior,760,A332,Business,ifl_300_cabin,76.0
1,Aircraft interior,641,A332,Economy,ifl_300_cabin,64.1
2,Aircraft interior,685,A333,Business,ifl_300_cabin,68.5
3,Aircraft interior,668,A333,Economy,ifl_300_cabin,66.8
4,Aircraft interior,620,A333,Premium EC,ifl_300_cabin,62.0
...,...,...,...,...,...,...
173,Wi-Fi,484,A333,Economy,ifl_600_wifi,48.4
174,Wi-Fi,531,A333,Premium EC,ifl_600_wifi,53.1
175,Wi-Fi,551,A350,Business,ifl_600_wifi,55.1
176,Wi-Fi,474,A350,Economy,ifl_600_wifi,47.4


In [5]:
# Creamos una función para transformar la tabla original en el formato solicitado por el usuario
def transform_to_touchpoint_satisfaction(df):
    # Creamos una columna combinada con touchpoint y '_satisfaction'
    df['touchpoint_satisfaction'] = df['survey_maritz_name'] + '_satisfaction'
    
    # Pivotamos el DataFrame para que cada touchpoint tenga su propia columna
    transformed_df = df.pivot_table(
        index=['Cabin', 'Model'],
        columns='touchpoint_satisfaction',
        values='S_Satisfaction_label',
        aggfunc='first'
    ).reset_index()

    # Aplanamos el MultiIndex de las columnas si es necesario
    transformed_df.columns.name = None
    
    return transformed_df

# Ejecutamos la función con el DataFrame original
df = transform_to_touchpoint_satisfaction(data_df)

df = df.rename(columns={'load_factor_satisfaction': 'load_factor', 'ticket_price_satisfaction': 'ticket_price', 'Cabin': 'cabin_in_surveyed_flight'})

variable_names = [
    "ticket_price",
    "load_factor",
    "pun_100_punctuality_satisfaction",
    "bkg_200_journey_preparation_satisfaction",
    "pfl_100_checkin_satisfaction",
    "pfl_200_security_satisfaction",
    "pfl_300_lounge_satisfaction",
    "pfl_500_boarding_satisfaction",
    "ifl_300_cabin_satisfaction",
    "ifl_200_flight_crew_annoucements_satisfaction",
    "ifl_600_wifi_satisfaction",
    "ifl_500_ife_satisfaction",
    "ifl_400_food_drink_satisfaction",
    "ifl_100_cabin_crew_satisfaction",
    "arr_100_arrivals_satisfaction",
    "con_100_connections_satisfaction",
    "loy_200_loyalty_programme_satisfaction",
    "img_310_ease_contact_phone_satisfaction"
]

df = df[['cabin_in_surveyed_flight', 'Model'] + variable_names]

df['cabin_in_surveyed_flight'] = df['cabin_in_surveyed_flight'].replace('Premium EC', 'Premium Economy')


df['load_factor']=df['load_factor']/100
df

,cabin_in_surveyed_flight,Model,ticket_price,load_factor,pun_100_punctuality_satisfaction,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_satisfaction,pfl_200_security_satisfaction,pfl_300_lounge_satisfaction,pfl_500_boarding_satisfaction,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_satisfaction,ifl_500_ife_satisfaction,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_satisfaction,con_100_connections_satisfaction,loy_200_loyalty_programme_satisfaction,img_310_ease_contact_phone_satisfaction
0,Business,A332,1341.2,0.939,78.2,73.3,84.4,85.9,69.1,76.9,76.0,82.3,50.8,81.6,65.5,83.6,79.8,72.2,67.3,52.9
1,Business,A333,1189.8,0.889,78.6,71.6,80.7,78.9,72.6,72.1,68.5,82.2,44.5,55.6,64.8,82.5,74.1,55.1,65.7,54.0
2,Business,A350,1311.5,0.910,85.2,72.4,81.4,82.7,77.3,78.1,75.2,80.5,55.1,74.9,70.6,80.4,81.8,71.4,68.2,52.2
3,Business,A350 C,966.2,0.907,88.0,64.1,71.4,83.0,77.8,74.0,62.0,80.0,NaN,61.4,55.1,74.0,76.6,86.4,80.6,70.0
4,Economy,A332,495.8,0.895,79.9,70.6,76.2,82.4,56.8,76.6,64.1,79.9,47.0,77.0,58.3,75.8,78.5,70.0,62.7,53.2
5,Economy,A333,413.2,0.838,73.5,64.7,70.2,78.5,81.3,74.3,66.8,75.4,48.4,70.4,61.1,75.4,75.0,62.7,58.2,50.0
6,Economy,A350,613.7,0.891,81.3,70.0,75.3,83.7,60.0,74.5,68.7,82.7,47.4,76.7,59.3,75.8,79.7,69.6,64.9,49.5
7,Economy,A350 C,523.5,0.943,82.4,69.5,70.8,80.9,50.0,74.7,66.2,83.0,NaN,62.4,58.4,69.1,79.6,72.6,54.9,50.0
8,Premium Economy,A333,636.6,0.858,72.7,70.8,70.0,77.1,70.0,72.3,62.0,75.2,53.1,71.6,54.5,68.3,75.8,63.5,69.6,42.9
9,Premium Economy,A350,809.0,0.903,86.1,79.5,82.6,90.4,82.6,83.8,74.1,81.6,54.3,79.3,59.8,76.5,82.9,76.9,71.6,63.8


In [6]:
str_start_date = '2024-01-01'
str_end_date = '2024-01-31'

str_execution_date = '2024-10-07'

# Extract unique cabins and hauls from your DataFrame
cabins = ['Economy', 'Business', 'Premium Economy']
models = ['A332', 'A333', 'A350', 'A350 C']
hauls = ['LH']

# By default
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

# Use own bucket and prefix
s3_bucket_nps = 'iberia-data-lake'  # In this case: iberia-data-lake
s3_bucket_nps_prefix = 'customer/nps_explainability_model'  # In this case: sagemaker/sagemaker-template

s3_bucket_lf = 'ibdata-prod-ew1-s3-customer'
s3_bucket_lf_prefix = 'customer/load_factor_to_s3_nps_model'

# Environment (sbx or prod pipeline)
environment = "develop"
environment_ppl = "prod" if environment == "production" else "sbx"

# Config file
configuration = utils.read_config_data()

# Parameters
is_last_date = "0"  # Optional
model_version = configuration.get("MODEL_VERSION")
vertical = configuration.get("VERTICAL")


s3_path_read_nps = 'customer/nps_surveys/export_historic'
s3_path_read_lf = "customer/load_factor_to_s3_nps_model"
use_type = 'Client'


In [7]:
# Import your pipeline definition
from production.pipelines_code.pipeline import get_pipeline as pipeline_definition

pipeline_name = f"ibdata-{vertical}-{model_version.lower()}-{environment_ppl}-ppl"
base_job_prefix = pipeline_name

# Initialize the pipeline
pipe_definition = pipeline_definition(
    region=region,
    role=role,
    default_bucket='iberia-data-lake',
    default_bucket_prefix='customer/simulations/sbx/pipeline',
    pipeline_name=pipeline_name,
    base_job_prefix=base_job_prefix
)
pipe_definition.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-10-09-15-38-22-257/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-10-09-15-38-22-257/source/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-10-09-15-38-22-537/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-10-09-15-38-22-537/source/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://iberia-data-lake/customer/simulations/sbx/pipeline/ibdata-customer-simulations-sbx-ppl/fra-2024-10-09-15-38-22-663/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://iberia-data

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:077156906314:pipeline/ibdata-customer-simulations-sbx-ppl',
 'ResponseMetadata': {'RequestId': 'ae9a001b-f115-4afb-87d8-e7c5da8de672',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ae9a001b-f115-4afb-87d8-e7c5da8de672',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Wed, 09 Oct 2024 15:38:23 GMT'},
  'RetryAttempts': 0}}

In [8]:
for str_haul in hauls:
    for str_cabin in cabins:
        for str_model in models:
            
            s3_path_write = f"customer/simulations/sbx/Marco_sims/{str_cabin}/{str_model}"
            
            df_filtered = df[(df['cabin_in_surveyed_flight']==str_cabin) & (df['Model'] == str_model)]
            
            # Drop unnecessary columns
            df_targets = df_filtered.drop(columns=[
                'cabin_in_surveyed_flight', 'Model'
            ], errors='ignore')  # Use errors='ignore' in case 'NPS_weighted' is not present

            # Prepare df_targets as a CSV string without headers and remove newlines
            df_targets_str = df_targets.to_csv(index=False, header=False).strip().replace('\n', '')
            
            # Si el string tiene longitud cero, pasamos a la siguiente iteración del bucle
            if not df_targets_str:
                continue
            
            # This cell starts an execution of the previous pipeline created
            execution = pipe_definition.start(
                parallelism_config=ParallelismConfiguration(max_parallel_execution_steps=4).to_request(),
                parameters=dict(
                    str_execution_date=str_execution_date,
                    is_last_date=is_last_date,
                    s3_path_write=s3_path_write,
                    s3_path_read_nps=s3_path_read_nps,
                    s3_path_read_lf=s3_path_read_lf,
                    str_start_date=str_start_date,
                    str_end_date=str_end_date,
                    str_cabin = str_cabin,
                    str_haul = str_haul, 
                    str_model = str_model,
                    df_targets = df_targets_str,
                    use_type = use_type
                )
            )
            execution.describe()
            
            

In [9]:
str_end_date

'2024-01-31'